# modeling

In [2]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from util import dbhelper

In [1]:
## 모델링에 필요한 라이브러리 및 모듈 불러오기
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## 1.이진 분류 문제를 수행하는 '다:1' 구조의 LSTM 모델 설계 

In [32]:
# 하이퍼파라미터 설정
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(1, activation='sigmoid'))

2022-02-09 17:41:26.622229: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [33]:
# 검증 데이터 loss 증가 시 학습 조기 종료
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
# 검증 데이터 acc 향상 시에만 모델 저장
mc = ModelCheckpoint('model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         1447300   
                                                                 
 gru (GRU)                   (None, 128)               88320     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,535,749
Trainable params: 1,535,749
Non-trainable params: 0
_________________________________________________________________


## 2. 모델 학습

In [ ]:
# model fit
history = model.fit(X_train, y_train, 
                    epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
model.save('model.h5')

In [34]:
# 테스트 데이터셋으로 정확도 평가하기
loaded_model = load_model('model.h5')
print("\n 테스트 정확도: %.5f" % (loaded_model.evaluate(X_test, y_test)[1]))

1398/1398 [==============================] - 82s 58ms/step - loss: 0.4948 - acc: 0.7815

 테스트 정확도: 0.78149
